### Imports

In [205]:
#importação de bibliotecas
import py_dss_interface
import random
import functions

#localização do arquivo
arquivo = r"C:\Users\leonardo.simoes\OneDrive - Sistema FIEB\centro_comp\host_cap\circuito\ckt5\Master_ckt5.dss"

#criação do objeto para referenciar ao OpenDSS
dss = py_dss_interface.DSSDLL()

OpenDSS Started successfully! 
OpenDSS Version 9.4.0.1 (64-bit build); License Status: Open 




### Compilar

In [206]:
#limpar a memoria e compilar o arquivo
dss.text("clear")
dss.text(f"Compile [{arquivo}]")

''

### Variáveis

In [207]:
#tensão padronizada do equivalente de thevenin
thevenin_pu = 1.045
#percentual de redução da carga
load_mult = 0.2
#fator de potencia
pf=0.92
#percentual de barras que vão ser utilizadas
q_perc_barras = 0.8
#passo de crescimento da potência em kva
p_step = 5 #kva
#....duvida
kva_to_kw = 1
#Valor inteiro para padronizar a randomização da distribuição de fotovoltaicos
valor_seed = 114

### Parte 1

In [208]:
#Editando impedancia para um valor pequeno 
#de elementos que ficam próximos a fonte, a fim de manter a tensão de entrada sem variações
dss.text("edit Reactor.MDV_SUB_1_HSB x=0.0000001 r=0.0000001")
dss.text(r"edit Transformer.MDV_SUB_1 %loadloss=0.000001 xhl=0.0000001")
#modificando a tensão pu da fonte
dss.text(f"edit Vsource.SOURCE pu={thevenin_pu}")
#redução da carga para uma porcentagem da instalada, a fim de aumentar a potência fluindo nas linhas 
dss.text(f"set loadmult = {load_mult}")

dss.solution_solve()

#Plotar perfil de tensão
dss.text("Plot Profile Phases = All")
#armazenando valores de tensão, p e q do circuito
tensoes = dss.circuit_all_bus_vmag_pu()
thev_kw = -dss.circuit_total_power()[0]
thev_kvar = -dss.circuit_total_power()[1]

### Parte 2

In [209]:
#todas as barras do circuito trifásicas de média tensão
barras_hc_geral = list()
#tensoes associadas as barras do circuito
barras_kv = dict()
#todas as barras
barras = dss.circuit_all_bus_names()

#loop para mapear as barras
for barra in barras:
    #setar a barra que vai ser avaliada
    dss.circuit_set_active_bus(barra)
    #mapeando a tensão da barra
    barra_kv = dss.bus_kv_base()
    #avaliando o número de fases
    num_fases = len(dss.bus_nodes())
    
    #condicional para avaliar se são trifásicas e de média tensão
    if barra_kv > 1.0 and num_fases == 3 and barra!= 'sourcebus':
        barras_hc_geral.append(barra)
        barras_kv[barra] = barra_kv

#selecionando um percentual das barras para fazer HC
random.seed(valor_seed)        
barras_hc = random.sample(barras_hc_geral, int(q_perc_barras*len(barras_hc_geral)))

#aplicando um sistema FV nas barras que foram escolhidas
for barra in barras_hc:
    functions.define_3ph_pvsystem(dss=dss, bus=barra, kv=barras_kv[barra], kva=kva_to_kw * p_step, pmpp = p_step)
    functions.add_bus_marker(dss=dss, bus=barra, color="red", size_marker=4)

#interpolando os pontos de coordenada para adicionar as coordenadas as barras que estão sem
dss.text("interpolate")
#calculando o fluxo
dss.solution_solve()
#plot do SEP
dss.text("plot circuit Power max=2000 n n C1=$00FF0000")

'C:\\Users\\leonardo.simoes\\OneDrive - Sistema FIEB\\centro_comp\\host_cap\\circuito\\ckt5\\ckt5_Power.DSV'

### Parte 3

In [210]:
#flags para indicar se há violação na rede
sobretensao = False
sobrecarga = False
#limite de tensao avaliado
v_limite = 1.05

#numero de iterações
i=0
#loop de cálculo do HC
while not sobretensao and not sobrecarga and i<1000:
    #incremento das iterações
    i += 1
    #aumento do sistema fotovoltaico de acordo com o passo definido
    functions.increment_pv_size(dss=dss, p_step=p_step, kva_to_kw=kva_to_kw, pf=pf, i=i)
    #calculo do fluxo
    dss.solution_solve()
    #armazenamento das tensoes
    tensoes = dss.circuit_all_bus_vmag_pu()
    #avaliação do ponto que possui a maior tensão
    v_max = max(tensoes)
    
    #condicional para avaliar a existência de sobretensão
    if v_max > v_limite:
        sobretensao = True

    #lógica para avaliar sobrecarga 
    #Selecionada a primeira linha do circuito
    dss.lines_first()
    #loop para varrer todas as linhas do circuito
    for _ in range(dss.lines_count()):
        #Selecionar a linha que vai ser avaliada
        dss.circuit_set_active_element(f"line.{dss.lines_read_name()}")
        #armazenar a corrente atual da linha
        i_linha = dss.cktelement_currents_mag_ang()
        #armazenar a corrente nominal da linha
        i_nom_linha = dss.lines_read_norm_amps()
        
        #avaliar se a corrente atual é maior que a corrente nominal
        #se for, há sobrecarga
        if (max(i_linha[0:12:2]) / i_nom_linha) > 1:
            sobrecarga = True
            break
    
        #ir para a próxima linha    
        dss.lines_next()

#Selecionar o valor anterior do fotovoltaico, para não ter violação
functions.increment_pv_size(dss=dss, p_step=p_step, kva_to_kw=kva_to_kw, pf=pf, i=i-1)
#calculo do fluxo
dss.solution_solve()

# potencia step * quantidade de iterações (valor da potência final) * numero de barras que foram aplicados os fv
nivel_penetracao = (i-1) * len(barras_hc) * p_step

#medindo a entrada do circuito com o sistema fv
thev_kw_com_fv = -1 * dss.circuit_total_power()[0]
thev_kvar_com_fv = -1 * dss.circuit_total_power()[1]
#todas as tensões nodais do circuito
v_pu = dss.circuit_all_bus_vmag_pu()
#armazenando a tensão máxima e mínima
tensao_min = min(v_pu)
tensao_max = max(v_pu)
#potências do fv ativa e reativa
fv_kw, fv_kvar = functions.get_total_pv_powers(dss)

### Resultados

In [211]:
nivel_penetracao, fv_kw, fv_kvar

(1640, 1508.8000006311945, 642.7461091003931)